### Multi-Agent Research Workflow with AgentWorkflow

## Setup

In [2]:
%pip install langchain
%pip install langchain groq


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

import os
os.environ["GROQ_API_KEY"] = "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"

In [4]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGroq(
    model_name="llama3-70b-8192",  # u otro modelo compatible, como llama3-8b-8192
)


In [5]:
%pip install langchain-community


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## System Design

In [6]:
%pip install -U duckduckgo-search

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_core.prompts import ChatPromptTemplate

research_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research assistant. Find detailed information about the topic."),
    ("user", "{input}")
])

write_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional writer. Summarize the following research."),
    ("user", "{input}")
])

review_prompt = ChatPromptTemplate.from_template("""
Eres un crítico editorial experimentado. Tu tarea es revisar un texto escrito por otro agente. 
Evalúa si es claro, coherente, correcto, y profesional.

Texto a revisar:
{output_text}

Tu revisión debe incluir:
1. Una crítica del contenido.
2. Correcciones sugeridas (si las hay).
3. Una versión final revisada del texto (si es necesario).
""")

In [15]:
# Herramienta del agente revisor
def review_output_agent(input_text: str) -> str:
    prompt = review_prompt.format(output_text=input_text)
    return llm.invoke(prompt)

def write_prompt_agent(input_text: str) -> str:
    prompt = write_prompt.format(input=input_text)
    return llm.invoke(prompt)

In [16]:
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for answering questions about current events or general world knowledge"
)

write_tool = Tool(
    name="Write Tool",
    func=write_prompt_agent,
    description="Escribe un resumen profesional del texto de investigación."
)

review_tool = Tool(
    name="Review Tool",
    func=review_output_agent,
    description="Revisa y critica un texto generado por otro agente, proporcionando correcciones y una versión mejorada si es necesario."
)


## Running the Workflow

In [22]:
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# Paso 1: Agente de investigación
research_agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Paso 2: Agente de escritura
writing_agent = initialize_agent(
    tools=[write_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Paso 3: Agente de revisión
review_agent = initialize_agent(
    tools=[review_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Paso 4: Adaptar agentes a runnables con impresión
research_chain = RunnableLambda(lambda x: print_and_return("🔍 Research Output", research_agent.run(x)))
writing_chain = RunnableLambda(lambda x: print_and_return("✍️ Writing Output", writing_agent.run(x)))
review_chain = RunnableLambda(lambda x: print_and_return("✅ Review Output", review_agent.run(x)))

# Función auxiliar para imprimir y retornar
def print_and_return(label, result):
    print(f"\n{label}:\n{result}\n{'-'*40}")
    return result

# Paso 5: Secuencia completa
multi_agent_chain = RunnableSequence(
    first=research_chain,
    middle=[writing_chain],
    last=review_chain
)


In [23]:
output = multi_agent_chain.invoke({"input": "The economic impact of AI on developing countries"})
print(output)



> Entering new AgentExecutor chain...
I need to understand the impact of AI on developing countries from an economic perspective.

Action: Search
Action Input: "economic impact of AI on developing countries"
Observation: The paper is structured as follows: section 2 reviews the literature about the socio-economic impact of AI; ... showcasing a deep divide between developed and developing countries in global AI readiness. According to the 2021 AI Readiness Index, nearly 40% of 160 countries have published or are drafting national AI strategies, indicating that ... The AI revolution's stakes are high, making cooperation across borders essential. No nation can tackle these complex challenges alone. With proactive policies and international support, developing countries can shape the trajectory of AI and maximize its benefits, but they must seize this transformative opportunity now. This paper examines the uneven global impact of AI, highlighting how its effects will be a function of (i)